In [ ]:
import numpy as np
from ipynb.fs.full.Critical_Section import *
import re
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1) 

In [ ]:

'''
pass1 of the algorithm where the events are parsed, CS info is generated, Events are mapped with CS
'''
def pass1(filename):
    events, critical_section_list,n,lock_map = gen_event_cs2(filename)
    events_cs = generate_cs_for_events(events, critical_section_list)
    critical_section_list = modify_cs(critical_section_list)
    return events,critical_section_list,n,lock_map,events_cs

'''
Checks if the two entries are CP ordered. They must contain conflicting events.
'''
def is_cp(cs_index1,cs_index2,cs):
    if(len(list(cs[cs_index1].write_objects.intersection(cs[cs_index2].read_objects)))>0 or 
    len(list(cs[cs_index1].write_objects.intersection(cs[cs_index2].write_objects)))>0 or 
    len(list(cs[cs_index1].read_objects.intersection(cs[cs_index2].write_objects)))>0):
        return True
    
    
# TODO Mark CP and HB matrices at one point only  
'''
Marks 1 for each value of  matrix[arr1][arr2] 
'''
def mark_matrix(matrix,arr1,arr2,events):
    for val in arr1:
        for val2 in arr2:
            matrix[val][val2] = 1
            matrix = mark_po(matrix,events,val,val2)
    return matrix
    
'''
Make any modifications required to the critical_section_list. 
Making the events id sorted.
'''
def modify_cs(critical_section_list):
    for i in range(0,len(critical_section_list)):
        cs = critical_section_list[i]
        cs.event_idx = sorted(cs.event_idx)
        critical_section_list[i] = cs
    return critical_section_list
'''
Iter through the critical_section_list
'''
def iter_cs(critical_section_list):
    for cs in critical_section_list:
        print(cs)
        
'''
Checks if the events are not cp ordered and are conflicting 
'''
def detect_cp_race(cp_matrix,events):
    is_race_detected  = False
    for i in range(cp_matrix.shape[0]):
        for j in range(i+1,cp_matrix.shape[0]):
            if cp_matrix[i][j]==0 and are_events_conflicting(events[i],events[j]):
                print('Race on events:',i,j,' Variable name: ',events[i].var_name)
                is_race_detected = True
    if not is_race_detected:
        print('No Race')
        
'''
Checks for any conflicting events on different threads.

'''        
def are_events_conflicting(e1,e2):
    if(e1.tid!=e2.tid and
        ((e1.e_type==0 and e2.e_type==1) or
       (e1.e_type==1 and e2.e_type==0) or
       (e1.e_type==1 and e2.e_type==1)) and 
       e1.var_name==e2.var_name):

        return True
    else:
        return False
    
'''
Map event with their CS numbers
'''   
def generate_cs_for_events(events,critical_section_list):
    events_cs = np.zeros(len(events))
    iter_i = 0
    for cs in critical_section_list:
        event_ids = cs.event_idx
        for e in event_ids:
            events_cs[e] = iter_i
        iter_i = iter_i + 1
    return events_cs
'''
Print matrix with row and columns numbered
'''
def print_matrix(matrix_name):
    labels = np.arange(1,matrix_name.shape[0]+1,1)
    df = pd.DataFrame(matrix_name, columns=labels, index=labels)
    print(df)
    
'''
Pass2 which generates the cp matrix using rulea.
Also fills the HB matrix 
'''                
def pass2(events,critical_section_list,lock_map):
    hb_matrix,cp_matrix,cs_cp_matrix = generate_hb_cp_matrix_using_rule_a(events,critical_section_list,lock_map)
    return hb_matrix,cp_matrix,cs_cp_matrix


'''
Generate CP matrix using rule A in CP paper that the variables must be conflicting for each lock section. 
'''    
def generate_hb_cp_matrix_using_rule_a(events,critical_section_list,lock_map):   
    hb_matrix = np.zeros((len(events),len(events)))
    cp_matrix = np.zeros((len(events),len(events)))
    cs_cp_matrix = np.zeros((len(critical_section_list),len(critical_section_list)))
    for key, value in lock_map.items():
        for iter_var in range(len(value)):
            for inner_iter_var in range(iter_var+1,len(value)):
                hb_matrix = mark_matrix(hb_matrix,critical_section_list[value[iter_var]].event_idx,critical_section_list[value[inner_iter_var]].event_idx,events)
                if is_cp(value[iter_var],value[inner_iter_var],critical_section_list):
                    cp_matrix = mark_matrix(cp_matrix,critical_section_list[value[iter_var]].event_idx,critical_section_list[value[inner_iter_var]].event_idx,events)
                    cs_cp_matrix[value[iter_var]][value[inner_iter_var]]=1
    return hb_matrix,cp_matrix,cs_cp_matrix

'''
Check if matrices are equal
'''
def are_matrices_equal(m1,m2):
    return np.array_equal(m1,m2)
    
'''
Rule B computation for the CP where if events in a CS are CP ordered then those 2 CS are also CP ordered. 
'''
def compute_rule_b(hb_matrix,cp_matrix,events,critical_section_list,lock_map,event_cs,cs_cp_matrix):

    for key, value in lock_map.items():
        for iter_var in range(len(value)):
            iter_var_events = critical_section_list[value[iter_var]].event_idx
            # TODO: Add the list to the cs while iter_cs
            iter_var_events_list = list(iter_var_events)
            cs_start = iter_var_events_list[0]
            cs_end = iter_var_events_list[len(iter_var_events_list)-1]
            for inner_iter_var in range(iter_var+1,len(value)):
                if cs_cp_matrix[value[iter_var]][value[inner_iter_var]]==0:
                    # Only if there was no cp marked between 2 CS

                    inner_iter_var_events = critical_section_list[value[inner_iter_var]].event_idx
                    # TODO: Add the list to the cs while iter_cs
                    inner_iter_var_events_list = list(inner_iter_var_events)
                    cs2_start = inner_iter_var_events_list[0]
                    cs2_end = inner_iter_var_events_list[len(inner_iter_var_events_list)-1]
                    
                    should_cp_marked = False
                    for e1 in iter_var_events_list:
                        for e2 in inner_iter_var_events:
                            if cp_matrix[e1][e2] == True:
                                should_cp_marked = True
                                break
                    
                    if should_cp_marked:
                        cp_matrix = mark_matrix(cp_matrix,critical_section_list[value[iter_var]].event_idx,critical_section_list[value[inner_iter_var]].event_idx,events)
                        cs_cp_matrix[value[iter_var]][value[inner_iter_var]] = 1 
   
    return hb_matrix,cp_matrix,cs_cp_matrix    

'''
Rule c computation which looksk at both hb and cp relations.
'''
def compute_rule_c(hb_matrix,cp_matrix,events):
#     hb_matrix2 = np.copy(hb_matrix)
#     cp_matrix2 = np.copy(cp_matrix)
    # start from n - 1 
    for i in reversed(range(hb_matrix.shape[0]-1)):
        for j in (range(hb_matrix.shape[0],i,-1)):
    
              end = hb_matrix.shape[0]
              for k in range(end-1,j,-1):
                if hb_matrix[i][j] == 1 and cp_matrix[j][k] == 1:
                    hb_matrix[i][k] = 1
                    cp_matrix[i][k] = 1
                    hb_matrix  = mark_po(hb_matrix,events,i,k)
                    cp_matrix  = mark_po(cp_matrix,events,i,k)
                elif cp_matrix[i][j] == 1 and hb_matrix[j][k] == 1:
                    hb_matrix[i][k] = 1
                    cp_matrix[i][k] = 1  
                    hb_matrix  = mark_po(hb_matrix,events,i,k)
                    cp_matrix  = mark_po(cp_matrix,events,i,k)
                        
    return hb_matrix,cp_matrix 


def mark_po(matrix,events,e1,e2):
    tid1 = events[e1].tid
    tid2 = events[e2].tid
    
    for e3 in range(0,e1+1):
        if events[e3].tid == tid1:
            for e4 in range(e2,len(events)):
                if events[e4].tid == tid2:
                    matrix[e3][e4] = 1
    return matrix


def CSv1(filename):
    print('-'*100)
    print('Reading log file ' + filename)
    events, critical_section_list,no_of_threads,lock_map,event_cs = pass1(filename)
    # print(len(events))
    hb_matrix,cp_matrix,cs_cp_matrix = pass2(events,critical_section_list,lock_map)    
    i = 0  
    while 1:
        cp_change = False
        hb_change = False
        cp_matrix1 = np.copy(cp_matrix)
        hb_matrix1 = np.copy(hb_matrix)
        hb_matrix,cp_matrix,cs_cp_matrix = compute_rule_b(hb_matrix,cp_matrix,events,critical_section_list,lock_map,event_cs,cs_cp_matrix)    
        hb_matrix,cp_matrix = compute_rule_c(hb_matrix,cp_matrix,events)
        i = i +1 
        if  are_matrices_equal(cp_matrix1,cp_matrix) and  are_matrices_equal(hb_matrix1,hb_matrix):
            break      
    detect_cp_race(cp_matrix,events)    